Copyright © 2020 IUBH Internationale Hochschule

# Filter

## Visualisieren des Effekts von Filtern

### Tiefpass  /  Hochpass  / Bandpass

### Ein beliebiges Signal im Zeitbereich definieren und das Amplitudenspektrum aus der FFT anzeigen. Wählen Sie einen der Filter im Filtermenü aus, stellen Sie ihn in beliebiger Weise ein und sehen Sie die Filterwirkung auf das Originalsignal.

In [ ]:
# resourcen
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

#set backend for interactive toolbar
%matplotlib nbagg 

In [ ]:
#presets
MAX_Amplitude = 5
MAX_Frequenz = 1000
MAX_Phase = 180

fNoise = 780
timeBase = np.arange(0.0, 3.0, 1/10000)     # zeitvektor für  0...3 s, sample rate:  1/fsample (default = 10kHz)

In [ ]:
# funktionen definieren
#
def getSinus(amp,f,phase,time):
    return(amp*np.sin(2*np.pi*f*time + phase))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.square.html
def getRect(amp,f,time):
    return(amp*signal.square(2*np.pi*f*time))

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.sawtooth.html#scipy.signal.sawtooth
def getTri(f,time):
    return(signal.sawtooth(2*np.pi*f*time, width=0.5))


def calc_FFT(fSample, signal):
    #https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html#numpy.fft.fft
    # fft mit numpy.fft - routinens
    #
    # DFT für signal berechnen
    fftSig = np.fft.fft(signal)
    #
    # anzahl der punkte des komplexen fft-vektors
    n = len(fftSig)
    # frequenzen den einzelnen Punkten zuordnen
    freq = np.fft.fftfreq(n, 1/fSample)
    #
    # positiven Bereich des komplexen Spektrums auswaehlen
    # die Nullfrequenzkomponente in die Mitte des Spektrums verschieben.
    # - amplituden und frequenzen
    Y1_shift = np.fft.fftshift(fftSig)
    F1_shift = np.fft.fftshift(freq)
    #
    # indes des Nullpunktes bestimmen
    #
    iZero = int(np.ceil(n/2.0))
    #
    # amplituden und frequenzen von 0 ... n/2 auswaehlen
    Y1_pos = Y1_shift[iZero:-1]
    F1_pos = F1_shift[iZero:-1]
    #
    # amplitude normalisieren mit (2* 1/n) und real-Anteil bestimmen
    #
    reSpectrum = 2 * 1/n * np.abs(Y1_pos)
    return(reSpectrum , F1_pos)
#

In [ ]:
# Filter berechnen
#
# https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html
def LowPass_filter(fSample, fCutOff, order):
    nyq = 0.5 * fSample
    normCutOff = fCutOff/nyq
    b,a = signal.butter(order,normCutOff,btype="low")
    return(b,a)
#     
def HighPass_filter(fSample, fCutOff, order):
    nyq = 0.5 * fSample
    normCutOff = fCutOff/nyq
    b,a = signal.butter(order,normCutOff,btype="high")
    return(b,a)
#     
def BandPass_filter(fSample, order, fCutOff_1, fCutOff_2):
    nyq = 0.5 * fSample
    low = fCutOff_1 / nyq
    high = fCutOff_2 / nyq
    b,a = signal.butter(order, [low, high], btype="band")
    return(b,a)

In [ ]:
#interaktive Schalter
# 
# signal menu
#
signalForms = ['Sinus', 'Rechteck', 'Dreieck']
signal_dropdown = widgets.Dropdown(description='Signalform auswählen', options=signalForms, value='Sinus')
#Sliders
amp_1 = widgets.FloatSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}$_1:")
frq_1 = widgets.FloatSlider(min=0, max=MAX_Frequenz, value=10, description="$f$_1: in Hz")
phase_1 = widgets.FloatSlider(min=0, max=MAX_Phase, value=0, description='$\phi$_1: in °')
#
amp_2 = widgets.FloatSlider(min=0, max=MAX_Amplitude, value=1, description="$\hat{y}$_2:")
frq_2 = widgets.FloatSlider(min=0, max=MAX_Frequenz, value=10, description="$f$_2: in Hz")
phase_2 = widgets.FloatSlider(min=0, max=MAX_Phase, value=0, description='$\phi$_2: in °')
#
fSample = widgets.IntSlider(min=100, max=100000, value=10000, step=10, description='Abtastfrequenz')
#
#checkbox for noise overlay
chkNoise = widgets.Checkbox(description='Störsignal hinzufügen')

# filter menu
#
filterTypes = ['Tiefpass','Hochpass','Bandpass']
filter_dropdown = widgets.Dropdown(description='Filtertyp wählen', options=filterTypes, value='Tiefpass')
#
fCut_1 = widgets.IntSlider(min=1, max=(fSample.value/2 -1), value=50,step=5, description='Grenzfrequenz')
fCut_2 = widgets.IntSlider(min=1, max=(fSample.value/2 -1), value=200, step=5, description='(Bandpass): obere Grenzfrequenz')
#
#edit field for filter order
filtOrder = widgets.IntSlider(min=0, max=64, value=3, description='Filter Ordnung:')
#
# reset Schalter
BtnReset = widgets.Button(description='Reset',tooltip='Setze alle Werte zurück')

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(10,8))
#
# 1. Callback-Funktion definieren mit der die Figuren aktualisiert werden, wenn sich ein Stueerelement geändert hat
def update_view(*args):
    
    # Zeitvektor aktualisieren
    timeBase = np.arange(0, 3, 1/fSample.value)     # zeitvektor für  0...3 s, sample rate:  1/fsample (default = 10kHz)
    
    #
    #Abtastfrequenz muss mindestens doppelt so hoch wie die höchste vorkommende Frequenz sein
    #(Nyquist - Theorem)
    # andernfalls: IndexError bei der FFT-Berechnung
    #
    if ((frq_1.value > 50) | (frq_2.value > 50)):
        fSample.min = 2* frq_1.value
        if (frq_2.value > frq_1.value):
            fSample.min = 2* frq_2.value
    if (chkNoise.value == True):        
        fSample.min = 2* np.max([frq_1.value, frq_2.value, fNoise])

    # dropdowns
    #
    signaltype = signal_dropdown.value
    filtertype = filter_dropdown.value
    
    #-----------------------------------------------------------
    # Ausgangssignal nach der gewählten Signalform rechnen
    #
    if (signaltype == "Sinus"):
        # evtl. ausgeschaltene Steuerlemente wieder aktivieren
        amp_2.disabled = False
        frq_2.disabled = False
        phase_2.disabled = False
        chkNoise.disabled = False

        ySin1 = getSinus(amp_1.value, frq_1.value, phase_1.value, timeBase)
        ySin2 = getSinus(amp_2.value, frq_2.value, phase_2.value, timeBase)
        ySum = ySin1 + ySin2
        if (chkNoise.value == True): # das Störsignal hat ein feste Frequenz von 780Hz und eine Amplitude von 5
            ySum = ySin1 + ySin2 + (5*np.sin(2*np.pi*fNoise*timeBase))
    
    if (signaltype == "Rechteck"):
        yRect1 = getRect(amp_1.value, frq_1.value, timeBase)
        # Steuerelemente des 2. (Sinus)-Signals deaktivieren
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True
        #
        ySum = yRect1
        #
    if (signaltype == "Dreieck"):
        yTri = getTri(frq_1.value, timeBase)
        # Steuerelemente des 2. (Sinus)-Signals deaktivieren        
        #
        amp_2.disabled = True
        frq_2.disabled = True
        phase_2.disabled = True
        chkNoise.disabled = True
        #
        ySum = yTri
        
    #-----------------------------------------------------------   
    # gewählten Filter berechnen und Signal filtern
    #
    # evt. Steuerlemente deaktiveren
    if (filtertype != 'band-pass'):
        fCut_2.disabled = True
        
    if (filtertype == "Tiefpass"):
        #print("filter parameters {} fSample, {} fCutOff, {} order".format(fSample.value, fCut_1.value, filtOrder.value))
        tp_b, tp_a = LowPass_filter(fSample.value, fCut_1.value, filtOrder.value)        
        #print("filter parameters {} b, {} a".format(tp_b, tp_a))
        s = signal.lfilter(tp_b, tp_a, ySum)
        w, h = signal.freqz(tp_b, tp_a, worN=10000)
        fResponse = 0.5 * fSample.value * w/np.pi
    if (filtertype == "Hochpass"):
        hp_b, hp_a = HighPass_filter(fSample.value, fCut_1.value, filtOrder.value)
        s = signal.lfilter(hp_b, hp_a, ySum)
        w, h = signal.freqz(hp_b, hp_a, worN=10000)
        fResponse = 0.5 * fSample.value * w/np.pi
    if (filtertype == "Bandpass"):
        fCut_2.disabled = False
        # überwache dass f1 < f2:
        if fCut_1.value > fCut_2.value:
            temp = fCut_2.value
            fCut_2.value = fCut_1.value
            fCut_1.value = (temp - 100)
        #
        bp_b, bp_a = BandPass_filter(fSample.value, filtOrder.value, fCut_1.value, fCut_2.value)
        s = signal.lfilter(bp_b, bp_a, ySum)
        w, h = signal.freqz(bp_b, bp_a, worN=10000)
        fResponse = 0.5 * fSample.value * w/np.pi
    
    #-----------------------------------------------------------
    # FFT berechnen
    #
    # Amplitudenspektrum des original-Signals:
    #
    fftSig, frqSig = calc_FFT(fSample.value, ySum)
    #
    # Amplitudenspektrum des gefilterten-Signals:
    #
    fftFilt, frqFilt = calc_FFT(fSample.value, s)
    #
    # Rücktransformation
    # (hier wieder die komplexe DFT verwenden)
    #
    reOrig = np.fft.ifft(np.fft.fft(ySum))
    reSig = np.fft.ifft(np.fft.fft(s))
    
    #-----------------------------------------------------------
    # Figuren aktualisiern
    #
    # angezeigten Zeitbereich auf 3 signalperioden begrenzen:
    if (frq_1.value < 2):
        xEnd = 3
    if (frq_1.value > 1):
        xEnd = 3* 1/frq_1.value
    if (signaltype == 'Sinus'): 
        if ((frq_1.value > 0) & (frq_2.value > 0) & (frq_1.value > frq_2.value)):
            xEnd = 3* 1/frq_2.value
    #
    # FFT-diagramme begrenzen:
    xFFTEnd = len(fftSig) -1
    add_f = -1
    peaks,_ = signal.find_peaks(fftSig, height=(0.01, 10))
    if (np.count_nonzero(peaks) > 0):
        fftError = False
        xFFTEnd = np.max(peaks)
        add_f = (len(fftSig) - xFFTEnd)-1
        if add_f > 100:
            add_f = 100
    
    # fig_1: original-signal im zeitbereich
    #
    axes[0, 0].clear()
    axes[0, 0].set_title("original-signal im Zeitbereich")
    axes[0, 0].plot(timeBase, ySum, color='C0',label=r"$y(t)_{{{}}},f:{{{}}}Hz$".format(signaltype,frq_1.value))
    axes[0, 0].set_xlabel("Zeit /s")
    axes[0, 0].set_ylabel("Amplitude")
    axes[0, 0].set_xlim(timeBase[0], xEnd)
    axes[0, 0].legend(loc="best")
    axes[0, 0].grid(True)
    
    # fig_2: fft des original-signals
    #
    axes[0, 1].clear()
    axes[0, 1].set_title("FFT des original-signals")
    axes[0, 1].plot(frqSig, fftSig, color='C1')
    axes[0, 1].set_xlabel("f /Hz")
    axes[0, 1].set_ylabel("Amplitude")
    axes[0, 1].set_xlim(frqSig[0], frqSig[xFFTEnd+add_f])
    axes[0, 1].grid(True)
    
    # fig_3: inverse_fft der original_fft
    #
    axes[0, 2].clear()
    axes[0, 2].set_title("Rücktransformation aus Original-FFT")
    axes[0, 2].plot(timeBase[:len(reOrig.real)], reOrig.real, color='C4', label="rücktransformiert\n aus FFT_1" )
    axes[0, 2].set_xlabel("Zeit /s")
    axes[0, 2].set_ylabel("Amplitude")
    axes[0, 2].set_xlim(timeBase[0], xEnd)
    axes[0, 2].legend(loc='upper center')
    axes[0, 2].grid(True)
    
    
    # fig_4 ... fig6: gefiltertes signal
    #
    # höchste vorkommende Frequenz aus der FFT bestimmen:
    #  - X-Achse des FFT-Diagrams begrenzen
    #  - höchste vorkommende Frequenz + 100Hz
    #
    xLim4 = len(frqFilt)
    add4 = 0
    peaks2, _ = signal.find_peaks(fftFilt, height=(0.5, 10))
    if (np.count_nonzero(peaks2) > 0):
        fftError = True
        xLim4 = np.max(peaks)
        add4 = (len(frqFilt) - xLim4) -1
        if add4 > 100:
            add4 = 100
    
    # fig4: filtered signal
    axes[1, 0].clear()
    axes[1, 0].set_title("gefiltertes Signal im Zeitbereich")
    axes[1, 0].plot(timeBase,s, color='C2', label="filter: {}".format(filtertype))
    axes[1, 0].set_xlabel("Zeit /s")
    axes[1, 0].set_ylabel("Amplitude")
    axes[1, 0].set_xlim(timeBase[0], xEnd)
    axes[1, 0].legend(loc='best')
    axes[1, 0].grid(True)

    # fig_5: fft des gefilterten signals
    #
    axes[1, 1].clear()
    axes[1, 1].set_title("FFT des gefilterten Signals")
    axes[1, 1].plot(frqFilt, fftFilt, color='C3')
    axes[1, 1].set_xlabel("f /Hz")
    axes[1, 1].set_ylabel("Amplitude")
    if (xLim4+ add4) == len(frqFilt):
        axes[1, 1].set_xlim(frqFilt[0], frqFilt[-1])
    else:
        axes[1, 1].set_xlim(frqFilt[0], frqFilt[xLim4+add4])
    axes[1, 1].grid(True)
 
    # fig_6: inverse_fft aus gefilterter_fft 
    #
    axes[1, 2].clear()
    axes[1, 2].set_title("Rücktransformation aus der gefilterten-FFT")
    axes[1, 2].plot(timeBase[:len(reSig.real)], reSig.real, color='C5', label="rücktransformiert\n aus FFT_2" )
    axes[1, 2].set_xlabel("Zeit /s")
    axes[1, 2].set_ylabel("Amplitude")
    axes[1, 2].set_xlim(timeBase[0], xEnd)
    axes[1, 2].legend(loc='upper center')
    axes[1, 2].grid(True)
    
    # fig_7: Frequenzgang des Filters
    #
    axes[2, 0].clear()
    axes[2, 0].set_title("Frequenzgang des Filters")
    axes[2, 0].plot(fResponse, np.abs(h), 'b')
    axes[2, 0].plot(fCut_1.value, 0.5*np.sqrt(2), 'ko')
    axes[2, 0].set_xlabel("Frequenz /Hz")
    axes[2, 0].set_ylabel("Amplitude")
    axes[2, 0].axvline(fCut_1.value, color='k')
    axes[2, 0].set_xlim(0, fCut_1.value * 5)
    axes[2, 0].grid(True)
    if filtertype == "Bandpass":
        axes[2, 0].axvline(fCut_2.value, color='k')
        axes[2, 0].plot(fCut_2.value, 0.5*np.sqrt(2), 'ko')
        axes[2, 0].set_xlim(0, fCut_2.value * 5)
        
    fig.tight_layout()

    axes[2, 1].set_axis_off()
    axes[2, 2].set_axis_off()


def reset_all(*args):
    signal_dropdown.value = 'Sinus'
    filter_dropdown.value = 'Tiefpass'
    fSample.value = 10000
    amp_1.value = 1
    amp_2.value = 1
    frq_1.value = 10
    frq_2.value = 10
    phase_1.value = 0
    phase_2.value = 0
    chkNoise.value = False
    fCut_1.value = 50
    fCut_2.value = 200
    filtOrder.value = 3

    
#--------------------------------------------------------
# 2. die Callback-Funktion mit der Funktion 'observe' den Steuerelementen zuweisen
signal_dropdown.observe(update_view, 'value')
filter_dropdown.observe(update_view, 'value')
fSample.observe(update_view,'value')
amp_1.observe(update_view,'value')
amp_2.observe(update_view,'value')
frq_1.observe(update_view,'value')
frq_2.observe(update_view,'value')
phase_1.observe(update_view,'value')
phase_2.observe(update_view,'value')
chkNoise.observe(update_view,'value')
fCut_1.observe(update_view,'value')
fCut_2.observe(update_view,'value')
filtOrder.observe(update_view, 'value')
BtnReset.on_click(reset_all)

#--------------------------------------------------------
# Anwendung starten
#
# Diagramme einmal zeichnen
update_view()
#
#
# mit 'widgets.VBox / .HBox' die Steuerelemente arangieren
# box-layout definieren
box_layout = widgets.Layout(display='inline-flex',flex_flow='column',align_items='flex-start',width='90%')

signal_menu = widgets.VBox([widgets.HBox([signal_dropdown, chkNoise]), fSample, widgets.HBox([amp_1, amp_2]),\
                            widgets.HBox([frq_1, frq_2]),widgets.HBox([phase_1, phase_2])], layout = box_layout)
filter_menu = widgets.VBox([filter_dropdown, widgets.HBox([fCut_1, filtOrder]), fCut_2], layout = box_layout)

widgets.VBox([widgets.HTML(value="<h2>signal menu:</h2><br>"),signal_menu,\
              widgets.HTML(value="<h2>filter menu:</h2><br>"),filter_menu, BtnReset])

Copyright © 2020 IUBH Internationale Hochschule